In [1]:
import os
%pwd
os.chdir("../")
%pwd

'd:\\Data Science\\END to END Proj\\KidneyDiseaseMLOPS'

In [2]:
%pip install dagshub
import dagshub
dagshub.init(repo_owner='gowtham-dd', repo_name='KidneyDiseaseMLOPS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Accessing as gowtham-dd

Initialized MLflow to track repo "gowtham-dd/KidneyDiseaseMLOPS"

Repository gowtham-dd/KidneyDiseaseMLOPS initialized!

In [3]:
from dataclasses import dataclass
from typing import List
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    mlflow_uri: str
    batch_size: int
    target_metric: str
    all_params: dict  


In [4]:

from KidneyClassifier.utils.common import read_yaml, create_directories
from pathlib import Path
from KidneyClassifier.constant import*
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])    
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_evaluation  # or whatever group you use

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
        root_dir=Path(config.root_dir),
        test_data_path=Path(config.test_data_path),
        model_path=Path(config.model_path),
        metric_file_name=Path(config.metric_file_name),
        mlflow_uri=config.mlflow_uri,
        batch_size=params.batch_size,
        target_metric=params.target_metric,
        all_params=params  
        )


In [5]:
import numpy as np
import tensorflow as tf
import mlflow
from KidneyClassifier.utils.common import save_json
from KidneyClassifier import logger

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def _load_model(self):
        # Load Keras saved model, not generic TF SavedModel
        model = tf.keras.models.load_model(str(self.config.model_path))
        return model

    def evaluate_model(self):
        # Load test data
        test_data = np.load(self.config.test_data_path)
        X_test, y_test = test_data['X_test'], test_data['y_test']

        # Convert to float32 if needed
        if X_test.dtype == np.float64:
            X_test = X_test.astype(np.float32)

        # Load Keras model
        model = self._load_model()

        # Predict using Keras model
        y_pred_probs = model.predict(X_test, batch_size=self.config.batch_size)
        y_pred = np.argmax(y_pred_probs, axis=1)

        # Calculate metrics
        accuracy = np.mean(y_test == y_pred)
        precision = tf.keras.metrics.Precision()(y_test, y_pred).numpy()
        recall = tf.keras.metrics.Recall()(y_test, y_pred).numpy()
        f1 = 2 * (precision * recall) / (precision + recall + 1e-7)  # small epsilon to avoid div by zero

        metrics = {
            "accuracy": float(accuracy),
            "precision": float(precision),
            "recall": float(recall),
            "f1_score": float(f1)
        }

        save_json(path=self.config.metric_file_name, data=metrics)
        logger.info(f"Evaluation metrics: {metrics}")
        return metrics
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)

        try:
            with mlflow.start_run():
            # ✅ Log all parameters
                mlflow.log_params(self.config.all_params)

            # Evaluate and log metrics
                metrics = self.evaluate_model()
                mlflow.log_metrics(metrics)

            # Log model
                mlflow.tensorflow.log_model(
                model=tf.keras.models.load_model(self.config.model_path),
                artifact_path="kidney_model",
                registered_model_name="Kidney_SavedModel"
            )

                logger.info("MLflow logging completed successfully")

        except Exception as e:
            logger.error(f"MLflow logging failed: {e}")
            if mlflow.active_run():
                mlflow.end_run(status="FAILED")
            raise


In [6]:
from KidneyClassifier import logger
try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    model_evaluator = ModelEvaluation(config=eval_config)  # Now works correctly
    model_evaluator.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e

[2025-07-25 01:59:39,376: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-25 01:59:39,403: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-25 01:59:39,406: INFO: common: created directory at: artifacts]
[2025-07-25 01:59:39,410: INFO: common: created directory at: artifacts/model_evaluation]
127/127 [==============================] - 3s 20ms/step
[2025-07-25 01:59:47,113: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
[2025-07-25 01:59:47,119: INFO: 2308986648: Evaluation metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 0.9999999500000026}]


2025/07/25 01:59:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-07-25 01:59:51,800: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _update_step_xla while saving (showing 4 of 4). These functions will not be directly callable after loading.]
[2025-07-25 01:59:52,596: INFO: builder_impl: Assets written to: C:\Users\dell\AppData\Local\Temp\tmpqodju0f1\model\data\model\assets]


d:\Data Science\END to END Proj\KidneyDiseaseMLOPS\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'Kidney_SavedModel' already exists. Creating a new version of this model...
2025/07/25 02:00:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Kidney_SavedModel, version 4


[2025-07-25 02:00:27,257: INFO: 2308986648: MLflow logging completed successfully]


Created version '4' of model 'Kidney_SavedModel'.


In [7]:
if mlflow.active_run():
    mlflow.end_run()
